In [1]:
import transformers

#Set to avoid warning messages.
transformers.logging.set_verbosity_error()

## 02.03. Summarization with Pipelines

In [2]:
verbose_text ="""
Earth is the third planet from the Sun and the only astronomical object
known to harbor life.
While large volumes of water can be found
throughout the Solar System, only Earth sustains liquid surface water.
About 71% of Earth's surface is made up of the ocean, dwarfing
Earth's polar ice, lakes, and rivers.
The remaining 29% of Earth's
surface is land, consisting of continents and islands.
Earth's surface layer is formed of several slowly moving tectonic plates,
interacting to produce mountain ranges, volcanoes, and earthquakes.
Earth's liquid outer core generates the magnetic field that shapes Earth's
magnetosphere, deflecting destructive solar winds.
"""

verbose_text = verbose_text.replace("\n","")


In [3]:
from transformers import pipeline


extractive_summarizer = pipeline("summarization", # predefined pipeline
                                 min_length=10,
                                 max_length=100)

#Extractive summarization
extractive_summary=extractive_summarizer(verbose_text)

print(extractive_summary[0].get("summary_text"))



config.json:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

 Earth is the third planet from the Sun and the only astronomical object known to harbor life . About 71% of Earth's surface is made up of the ocean, dwarfing Earth's polar ice, lakes, and rivers . The remaining 29% of the surface is land, consisting of continents and islands .


In [ ]:
# checking the model configuration
print("Checkpoint used: ", extractive_summarizer.model.config)

Checkpoint used:  BartConfig {
  "_name_or_path": "sshleifer/distilbart-cnn-12-6",
  "_num_labels": 3,
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 12,
  "eos_token_id": 2,
  "extra_pos_embeddings": 2,
  "force_bos_token_to_be_generated": true,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "init_std": 0.02,
  "is_en

## 02.05 Evaluating with ROUGE

In [4]:
! pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.1 MB/s eta 0:00:00


In [6]:
! pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=ac3b68c4878eef5ba45d9a8e786f7c8b83ddb2e4312c6889bd547e02592a03b2
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [7]:
import evaluate

rouge_evaluator = evaluate.load("rouge")

# Test the sample

#Evaluate exact match strings
reference_text=["This is the same string"]
predict_text=["This is the same string"]

eval_results=rouge_evaluator.compute(predictions=predict_text,
                                     references=reference_text)
print("Results for Exact match",eval_results)

# Since the reference and predict are same we got all the scores as 1

Results for Exact match {'rouge1': 1.0, 'rouge2': 1.0, 'rougeL': 1.0, 'rougeLsum': 1.0}


In [8]:
#Evaluate no-match strings
reference_text=["This is the different string"]
predict_text=["Google can predict warm weather"]

eval_results=rouge_evaluator.compute(predictions=predict_text,
                                     references=reference_text)
print("\nResults for no match", eval_results)

# since there is no match between the reference and predict we got all the score as 0


Results for no match {'rouge1': 0.0, 'rouge2': 0.0, 'rougeL': 0.0, 'rougeLsum': 0.0}


In [9]:
#Evaluate summary
eval_results=rouge_evaluator.compute(
    predictions=[extractive_summary[0].get("summary_text")],
    references=[verbose_text])

print("\nResults for Summary generated", eval_results)

# for our text we got the rouge score of more than 60% for ROUGE 1, 2, 3 etc


Results for Summary generated {'rouge1': 0.6369426751592356, 'rouge2': 0.5935483870967743, 'rougeL': 0.624203821656051, 'rougeLsum': 0.624203821656051}
